# Initial


In [1]:
# import pandas as pd

# root_path = '/kaggle/input/child-mind-institute-problematic-internet-use/'
# data_train = pd.read_csv(f'{root_path}/train.csv')
# data_test = pd.read_csv(f'{root_path}/test.csv')



# Data Preprocessing

In [2]:
# from sklearn.preprocessing import LabelEncoder
# def handle_data_classifier(data):
#     df_encoded = data.copy()
#     for column in df_encoded.columns:
#         if df_encoded[column].dtype == 'object' or df_encoded[column].dtype.name == 'category':
#             le = LabelEncoder()
#             df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))
#         if df_encoded[column].dtype in ['int64', 'float64']: 
#             df_encoded[column].fillna(df_encoded[column].mean(), inplace=True)
#     return df_encoded

In [3]:
# print(data_test.columns)
# same_columns = list(set(data_train.columns).intersection(set(data_test.columns)))
# same_columns.remove('id')
# # drop row which dont have sii
# data_train = data_train[~data_train['sii'].isna()]
# print(data_train.columns)
# data_train = data_train.drop(columns=['id'])
# data_X = data_train[same_columns]
# data_y = data_train['sii']
# print(len(data_X.columns))
# # handle data which category type and Nan value
# data_X = handle_data_classifier(data_X)



# Train and test model

In [4]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)
# # X_train.columns

In [5]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# # Dự đoán và đánh giá
# y_pred = model.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))

# Handle data to submit

In [6]:

# len(data_test.columns)
# index = data_test['id']
# data_test = data_test.drop(columns=['id'])
# data_test = handle_data_classifier(data_test)
# ans = model.predict(data_test)
# series = pd.Series(ans)
# concatenated_dataframe = pd.concat([index, series], axis=1)
# concatenated_dataframe = pd.DataFrame(concatenated_dataframe)
# concatenated_dataframe = concatenated_dataframe.rename(columns = {0 : 'sii'})
# concatenated_dataframe

In [7]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

root_path = '/kaggle/input/child-mind-institute-problematic-internet-use/'
data_train = pd.read_csv(f'{root_path}/train.csv')
data_test = pd.read_csv(f'{root_path}/test.csv')

# print(data_test.columns)
same_columns = list(set(data_train.columns).intersection(set(data_test.columns)))
same_columns.remove('id')
# # drop row which dont have sii
data_train = data_train[~data_train['sii'].isna()]
# # print(data_train.columns)
data_train = data_train.drop(columns=['id'])
data_X = data_train[same_columns]
data_y = data_train['sii']
print(len(data_X.columns))

def handle_data_classifier(data) :
    encoded_data = data.copy()
    all_seasons = ['Spring', 'Summer', 'Fall', 'Winter']

    season_cols = encoded_data.select_dtypes(include=['object', 'category']).columns
    numeric_cols = encoded_data.select_dtypes(include=['int64', 'float64']).columns

    for col in numeric_cols:
        encoded_data.fillna({col : encoded_data[col].mean()}, inplace=True)

    for col in season_cols:
        one_hot = pd.get_dummies(encoded_data[col], prefix=col, dtype=int)
        one_hot = one_hot.reindex(columns=[f"{col}_{season}" for season in all_seasons], fill_value=0)
        encoded_data = pd.concat([encoded_data, one_hot], axis=1)
        encoded_data = encoded_data.drop(columns=[col])

    return encoded_data

data_X = handle_data_classifier(data_X)

print(np.bincount(data_y))

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(data_X, data_y)

print(np.bincount(y_train_resampled))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

from sklearn.cluster import KMeans

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_train)
print(np.bincount(clusters))

cluster_labels = []
for i in range(4):  # Loop over each cluster
    cluster_mask = (clusters == i)
    print(clusters == i)
    print(i, " ", np.bincount(y_train[cluster_mask]))
    most_common_label = np.bincount(y_train[cluster_mask]).argmax()
    cluster_labels.append(most_common_label)

print(cluster_labels)

new_clusters = kmeans.predict(X_test)
y_pred = np.array([cluster_labels[cluster] for cluster in new_clusters])

print("Accuracy:", accuracy_score(y_test, y_pred))

len(data_test.columns)
index = data_test['id']
data_test = data_test.drop(columns=['id'])
data_test = handle_data_classifier(data_test)
ans = kmeans.predict(data_test)
series = pd.Series(ans)
concatenated_dataframe = pd.concat([index, series], axis=1)
concatenated_dataframe = pd.DataFrame(concatenated_dataframe)
concatenated_dataframe = concatenated_dataframe.rename(columns = {0 : 'sii'})
concatenated_dataframe

58
[1594  730  378   34]
[1594 1594 1594 1594]


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[ 584 1124  576 2816]
[False False False ... False False False]
0   [298 162  81  43]
[False  True  True ... False False  True]
1   [519 343 193  69]
[False False False ... False False False]
2   [259 155 142  20]
[ True False False ...  True  True False]
3   [ 172  637  859 1148]
[0, 0, 0, 3]
Accuracy: 0.44357366771159873


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


,id,sii
0,00008ff9,3
1,000fd460,3
2,00105258,3
3,00115b9f,3
4,0016bb22,3
5,001f3379,3
6,0038ba98,3
7,0068a485,3
8,0069fbed,3
9,0083e397,3


In [8]:
concatenated_dataframe['sii'] = concatenated_dataframe['sii'].astype('int')
concatenated_dataframe.to_csv('/kaggle/working/submission.csv',index  = False)